In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
# Resaved the metacritic_movies.csv to get rid of commas in the numbers.
# Created schema for movies_t. Imported data into table just to make sure it worked, then deleted the table, and recreated
# to be an empty table.

In [ ]:
# load the movies data.
csv_file = "Metacritic Reviews/metacritic_movies.csv"
raw_movies_df = pd.read_csv(csv_file)
raw_movies_df.head()

In [ ]:
# Reviews data cleaning
# Previewed in Excel.
# Removed lines of invalid data that appears to have wrapped incorrectly.

# Need to fix text for the apostrophes that have been convered to â€™
# Did Find/Replace using text editor.
# 
# created file Cleaned_metacritic_reviews.csv

In [ ]:
# Load the reviews data.
csv_file = "Metacritic Reviews/Cleaned_metacritic_reviews.csv"
raw_reviews_df = pd.read_csv(csv_file, dtype={3:'str',4:'str'})
raw_reviews_df.head()

In [ ]:
## Cleanup complete

In [ ]:
## Addy's section

# Need to create table of movie posters urls from TheMovieDB.org
#
# Choice 1: Lookup table
#    Movie name, poster url

# Choice 2: Append a new column for poster url to movies_df

In [ ]:
## write out all df's to postgresql

In [ ]:
## test a movie lookup to search for a movie name, which should come back with:
#  movie name, all reviews for the movie, and movie poster url.

In [ ]:
import requests

In [ ]:
#url = 'https://api.themoviedb.org/3/movie/550?api_key=3d48b9204ca053957eb4375bb97a83c8'

In [ ]:
#keep
config_path = 'http://api.themoviedb.org/3/configuration?api_key={key}'
KEY = '3d48b9204ca053957eb4375bb97a83c8'
url = config_path.format(key=KEY)
r = requests.get(url)
config = r.json()
# config

In [ ]:
#keep
base_url = config['images']['base_url']
base_url

In [ ]:
# i queryed one of the movie using the title
# the url query returns an id 

In [ ]:
#keep 
temp = 'http://api.themoviedb.org/3/search/movie?query=Gavaga&api_key=3d48b9204ca053957eb4375bb97a83c8'
r = requests.get(temp)
response = r.json()
response['results'][0]['poster_path']

In [ ]:
# keep
base_url = 'http://image.tmdb.org/t/p/'
size = 'w500'
rel_path = '/mgwZvl0DwS1GIborSGlpTGomrJD.jpg'

In [ ]:
# keep
url = base_url + size + rel_path
url

In [ ]:
# save url to new lookup dataframe that has 2 columns: movie title, url


In [ ]:
base_url = 'http://image.tmdb.org/t/p/'
size = 'w500'

url_list = []
for title in raw_movies_df.movie_title:
    #print(title)
    temp = 'http://api.themoviedb.org/3/search/movie?query="' + title + '"&api_key=3d48b9204ca053957eb4375bb97a83c8'
    #print(temp)
    try:
        r = requests.get(temp)
        response = r.json()
        url = base_url + size + response['results'][0]['poster_path']
        url_list.append(url)
        print(url_list)
        #read json into a list
        #movie_url.append(response_json, [url])
    except:
        print('movie url not found. Skipping...')

print('-----------------------------')
print('Data Retrieval Complete')
print('-----------------------------')
    

In [ ]:
# url_df = pd.DataFrame(url_list)
# url_df

In [ ]:
# New DataFrame with selected colum(movie_title) from movies_df
# movie_title_df = raw_movies_df[['movie_title']].copy
# movie_title__df

In [ ]:
# result = pd.concat([movie_title_df, url_df],axis= , sort=True)

In [ ]:
# reference
# for title in raw_movies_df.movie_title:
#     #print(title)
#     temp = 'http://api.themoviedb.org/3/search/movie?query="' + title + '"&api_key=3d48b9204ca053957eb4375bb97a83c8'
#     #print(temp)
#     r = requests.get(temp)
#     response = r.json()
#     url = base_url + size + response['results'][0]['poster_path']
#     print(url)